In [1]:
!nvidia-smi

Mon Jan  3 09:06:52 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from psutil import *
cpu_count()

4

In [3]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2199.998
cache size	: 56320 KB
physical id	: 0
siblings	: 4
core id		: 0
cpu cores	: 2
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa
bogomips	: 4399.99
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 b

In [4]:
import gym
import numpy as np
import random

import plotly.graph_objs as go
import pickle as p

In [5]:
env = gym.make('FrozenLake-v0', is_slippery=False)

In [6]:
state = env.reset()
print('  (Start)')
env.render()

while True:    
    action = env.action_space.sample() 
    state, reward, done, info = env.step(action)
    print('\n')
    env.render()
    print(f'State = {state}, Reward = {reward}, Done = {done}')

    if done: 
      break;

env.close()

  (Start)

SFFF
FHFH
FFFH
HFFG


  (Up)
SFFF
FHFH
FFFH
HFFG
State = 0, Reward = 0.0, Done = False


  (Down)
SFFF
FHFH
FFFH
HFFG
State = 4, Reward = 0.0, Done = False


  (Right)
SFFF
FHFH
FFFH
HFFG
State = 5, Reward = 0.0, Done = True


In [7]:
Q = np.zeros([env.observation_space.n, env.action_space.n])

print(Q.shape)
print(Q)

(16, 4)
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [8]:
learning_rate = 0.8
gamma = 0.95
num_episode = 1000

In [9]:
reward_list = []

for i in range(num_episode):
    state = env.reset()
    sum_reward = 0

    while True:
        action = np.argmax(Q[state,:] + np.random.randn(1, env.action_space.n)*(1./(i+1)))

        next_state, reward, done, info = env.step(action)
        Q[state,action] = Q[state,action] + learning_rate*(reward + gamma*np.max(Q[next_state,:]))
        
        sum_reward += reward
        state = next_state
        
        if done == True:
            break
  
    reward_list.append(sum_reward)

In [10]:
h1 = go.Scatter(y=reward_list, 
                    mode="lines", line=dict(
                    width=2,
                    color='blue'),
                    name="reward"
                   )

data = [h1]
layout1 = go.Layout(title='Total Reward',
                   xaxis=dict(title='Episode'),
                   yaxis=dict(title=''))
fig1 = go.Figure(data = data, layout=layout1)
fig1.show(renderer="colab")

In [11]:
print("Final Q-Table")
print(Q)

Final Q-Table
[[0.00000000e+00 1.42376245e+14 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.26297864e+12 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 9.32574159e+09 0.00000000e+00]
 [0.00000000e+00 5.50264410e+07 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.21600000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 2.43238304e+05 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 7.16000000e+02 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.0000000

In [12]:
state = env.reset()
print('  (Start)')
env.render()

while True:    
    action = np.argmax(Q[state,:])
    state, reward, done, info = env.step(action)
    print('\n')
    env.render()
    print(f'State = {state}, Reward = {reward}, Done = {done}')

    if done: 
      break;

env.close()

  (Start)

SFFF
FHFH
FFFH
HFFG


  (Down)
SFFF
FHFH
FFFH
HFFG
State = 4, Reward = 0.0, Done = False


  (Down)
SFFF
FHFH
FFFH
HFFG
State = 8, Reward = 0.0, Done = False


  (Right)
SFFF
FHFH
FFFH
HFFG
State = 9, Reward = 0.0, Done = False


  (Down)
SFFF
FHFH
FFFH
HFFG
State = 13, Reward = 0.0, Done = False


  (Right)
SFFF
FHFH
FFFH
HFFG
State = 14, Reward = 0.0, Done = False


  (Right)
SFFF
FHFH
FFFH
HFFG
State = 15, Reward = 1.0, Done = True
